## LSPR: Pygbe vs analytical (experimental data)

We run pygbe and the analytical solution for the Cext calculation for the cases we have experimental data. We compare the error between these runs and analyze the results. 

We run both silver and gold, using the data from [Johnson and Christy, 1972](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.6.4370). We run meshes from 2K elements of a nanoparticle of radius 10nm.

### Silver



In [2]:
import numpy

In [3]:
#import data
lambda_s , n_s , k_s = numpy.loadtxt('gold_silver_water_raw-data/silver_JC72.txt', unpack=True)
lambda_s = lambda_s*1e3 #To convert to nm

### Water interpolation data for lambda_silver

We need to obtain the refractive index for the same wavelengths we have data for the silver case. We will save the data on a file for futere purposes. 

In [8]:
from data_analysis_helper import linear_interp

In [9]:
lambda_w , n_w , k_w = numpy.loadtxt('gold_silver_water_raw-data/water_HQ72.txt', unpack=True)
lambda_w = lambda_w*1e3 #To convert to nm

In [10]:
#linear interpolation
water_real_linear , water_imag_linear = linear_interp(lambda_w, n_w, k_w)

In [17]:
print(min(lambda_s))
print(min(lambda_w))
print(max(lambda_s))
print(max(lambda_w))

187.9
200.0
1937.0
2000.0


Our water data starts at a wavelength of 200 nm, therefore we can interpolate values smaller than that. So we will strip out the values for the silver wavelengths that are not in the water interpolation range. 

In [16]:
numpy.where(lambda_s<200.0)

(array([0, 1, 2, 3]),)

In [30]:
lambda_ws = lambda_s[4:]

In [42]:
n_s = n_s[4:]
k_s = k_s[4:]

In [43]:
#lambda_sw

In [44]:
n_ws = water_real_linear(lambda_sw)
k_ws = water_imag_linear(lambda_sw)

In [45]:
numpy.savetxt('interp_data/water_silv_nm.txt', list(zip(lambda_sw, n_ws, k_ws)), fmt='%1.3f')

In [46]:
refrac_silver = n_s + 1j * k_s
refrac_water  = n_ws + 1j * k_ws

In [47]:
diel_silver = refrac_silver * refrac_silver 
diel_water  = refrac_water * refrac_water

In [50]:
print(lambda_ws[23], diel_silver[23] )
print(lambda_ws[23], diel_water[23] )

381.5 (-3.471996+0.1864j)
381.5 (1.7968866304+8.240198656e-09j)
